# Загрузка

In [1]:
import requests
from urllib.parse import urlencode
from io import BytesIO
from zipfile import ZipFile

def download(public_key):
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'

    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']
    response = requests.get(download_url)

    dist_path = '/home/jupyter/datasphere/project/олени'
    zipfile = ZipFile(BytesIO(response.content))
    zipfile.extractall(path=dist_path)

In [ ]:
download('https://disk.yandex.ru/d/CR1nOiUCHtTExA') # поменять ссылку в итоговом варианте

# Обработка 

Из всех классов были удалены дубликаты с помощью программы на пк и вручную удалены неподходящие изображения   
Выделяем оленей в bounding boxes на train-датасете с помощью [предобученной модели на нескольких видах оленей](https://universe.roboflow.com/jeonjuuniv/deer-hqp4i) через api Roboflow

In [ ]:
from roboflow import Roboflow
import supervision as sv
import cv2
from tqdm import tqdm
import glob
import pandas as pd
import os

rf = Roboflow(api_key="VddTUJz8hUIPlIFyhwFY")
project = rf.workspace().project("deer-hqp4i")
model = project.version(1).model

In [ ]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

## Кабарга

Создаем папки data/images, data/labels; все данные про Кабаргу перенмосим в data/images

In [ ]:
!unzip "/home/jupyter/datasphere/project/олени/Кабарга.zip" -d "/home/jupyter/datasphere/project/data/images"

In [6]:
kab_paths = glob.glob('data/images/Кабарга/*')

In [7]:
for i in kab_paths:
    os.rename(i, '/home/jupyter/datasphere/project/data/images/' + i.split('/')[-1])

Удаляем папку "Кабарга"

In [39]:
images = glob.glob('data/images/*')
dest_path = 'data/labels'

In [ ]:
images_path = images[:]

In [48]:
srange_images = []
for path in tqdm(images_paths):
    if path.endswith('.csv'):
        continue
    result = model.predict(path, confidence=30, overlap=50).json()
    name = path.split('/')[-1].split('.')[0]
    if len(result['predictions']) > 0:
        fp = open(f'{dest_path}/{name}.txt', 'w')
        preds = []
        for pred in result['predictions']:
            preds.append(f"0 {result['predictions'][0]['x']/float(result['image']['width'])} {result['predictions'][0]['y']/float(result['image']['height'])} {result['predictions'][0]['width']/float(result['image']['width'])} {result['predictions'][0]['height']/float(result['image']['height'])}\n")
        fp.writelines(preds)
        fp.close()
    else:
        srange_images.append(path)
        

100%|██████████| 1861/1861 [29:12<00:00,  1.06it/s]


In [49]:
fp = open(f'broken_kabara.txt', 'w')
fp.writelines(map(lambda x: x+'\n',srange_images))
fp.close()

In [52]:
len(srange_images), len(images) - len(glob.glob('/home/jupyter/datasphere/project/data/labels/*'))

(288, 375)

In [53]:
fp = open(f'broken_kabara.txt', 'r')
image_paths = list(map(lambda x: x[:-1], fp.readlines()))
fp.close()
dest_path = 'data/labels'

In [54]:
srange_images = []
for path in tqdm(image_paths):
    if path.endswith('.csv'):
        continue
    result = model.predict(path, confidence=20, overlap=50).json()
    name = path.split('/')[-1].split('.')[0]
    if len(result['predictions']) > 0:
        fp = open(f'{dest_path}/{name}.txt', 'w')
        preds = []
        for pred in result['predictions']:
            preds.append(f"0 {result['predictions'][0]['x']/float(result['image']['width'])} {result['predictions'][0]['y']/float(result['image']['height'])} {result['predictions'][0]['width']/float(result['image']['width'])} {result['predictions'][0]['height']/float(result['image']['height'])}\n")
        fp.writelines(preds)
        fp.close()
    else:
        srange_images.append(path)
        

100%|██████████| 288/288 [04:52<00:00,  1.02s/it]


In [55]:
fp = open(f'broken_kabara.txt', 'w')
fp.writelines(map(lambda x: x+'\n',srange_images))
fp.close()

In [57]:
df = pd.read_csv('/home/jupyter/datasphere/project/data/images/Кабарга.csv')
df['x_c'] = (df['bbox_x'] + df['bbox_width'] /2).apply(lambda x: int(x))
df['y_c'] = (df['bbox_y'] +df['bbox_height'] /2).apply(lambda x: int(x))
df['image_name'] = 'data/images/' + df['image_name']


In [58]:
for ind,row in df.iterrows():
    name = path.split('/')[-1].split('.')[0]
    fp = open(f'{dest_path}/{name}.txt', 'w')
    fp.write(f"0 {row['x_c']} {row['y_c']} {int(row['bbox_width'])} {int(row['bbox_height'])}")
    fp.close()

In [59]:
srange_images = list(set(srange_images) - set(df['image_name']))

Вручную размечаем оставшиеся из srange_images (Кабарга часть 2.zip)

In [63]:
for i in srange_images:
    os.remove(i)
os.remove('/home/jupyter/datasphere/project/data/images/Кабарга.csv')

In [ ]:
!unzip "/home/jupyter/datasphere/project/олени/Кабарга_часть2.zip"

In [65]:
images = glob.glob('/home/jupyter/datasphere/project/train/images/*')
labels = glob.glob('/home/jupyter/datasphere/project/train/labels/*')

In [66]:
for i in range(len(images)):
    os.rename(images[i], '/home/jupyter/datasphere/project/data/images/' + images[i].split('/')[-1])

In [67]:
for i in range(len(labels)):
    os.rename(labels[i], '/home/jupyter/datasphere/project/data/labels/' + labels[i].split('/')[-1])

In [68]:
images = glob.glob('/home/jupyter/datasphere/project/data/images/*')
labels = glob.glob('/home/jupyter/datasphere/project/data/labels/*')

In [69]:
for i in range(len(images)):
    os.rename(images[i], images[i].rstrip(images[i].split('/')[-1])+'cabarga_'+images[i].split('/')[-1]) # добавляем префикс для класса 2

In [70]:
for i in range(len(labels)):
    os.rename(labels[i], labels[i].rstrip(labels[i].split('/')[-1])+'cabarga_'+labels[i].split('/')[-1]) # добавляем префикс для класса 2

In [71]:
images = glob.glob('/home/jupyter/datasphere/project/data/images/*')
labels = glob.glob('/home/jupyter/datasphere/project/data/labels/*')

In [72]:
len(labels), len(images)

(2270, 2393)

In [ ]:
with open('/home/jupyter/datasphere/project/kabarga_del.txt') as f:
    ls = list(map(str.rstrip, f.readlines()))

In [ ]:
import os
for i in ls:
    try:
        os.remove("/home/jupyter/datasphere/project/data/labels/" + i.rstrip(i.split('.')[-1])+'txt')
    except Exception:
        print(1)

In [ ]:
for i in ls:
    try:
        os.remove("/home/jupyter/datasphere/project/data/images/" + i)
    except Exception:
        print(1)

## Косуля

Создаем папки ndata/images, ndata/labels

In [ ]:
!unzip "/home/jupyter/datasphere/project/олени/Косуля.zip" -d "/home/jupyter/datasphere/project/ndata/images"

In [78]:
kab_paths = glob.glob('ndata/images/Косуля/*')

In [79]:
for i in kab_paths:
    os.rename(i, '/home/jupyter/datasphere/project/ndata/images/' + i.split('/')[-1])

Удаляем папку "Косуля"

In [88]:
images = glob.glob('ndata/images/*')
dest_path = 'ndata/labels'

In [96]:
images_paths = images[:]

In [97]:
srange_images = []
for path in tqdm(images_paths):
    if path.endswith('.csv'):
        continue
    result = model.predict(path, confidence=30, overlap=50).json()
    name = path.split('/')[-1].split('.')[0]
    if len(result['predictions']) > 0:
        fp = open(f'{dest_path}/{name}.txt', 'w')
        preds = []
        for pred in result['predictions']:
            preds.append(f"1 {result['predictions'][0]['x']/float(result['image']['width'])} {result['predictions'][0]['y']/float(result['image']['height'])} {result['predictions'][0]['width']/float(result['image']['width'])} {result['predictions'][0]['height']/float(result['image']['height'])}\n")
        fp.writelines(preds)
        fp.close()
    else:
        srange_images.append(path)

100%|██████████| 759/759 [12:34<00:00,  1.01it/s]


In [98]:
fp = open(f'broken_kosul.txt', 'w')
fp.writelines(map(lambda x: x+'\n',srange_images))
fp.close()

In [102]:
len(srange_images)

2

In [103]:
fp = open(f'broken_kosul.txt', 'r')
image_paths = list(map(lambda x: x[:-1], fp.readlines()))
fp.close()
dest_path = 'ndata/labels'

In [107]:
srange_images = []
for path in tqdm(image_paths):
    if path.endswith('.csv'):
        continue
    result = model.predict(path, confidence=20, overlap=50).json()
    name = path.split('/')[-1].split('.')[0]
    if len(result['predictions']) > 0:
        fp = open(f'{dest_path}/{name}.txt', 'w')
        preds = []
        for pred in result['predictions']:
            preds.append(f"1 {result['predictions'][0]['x']/float(result['image']['width'])} {result['predictions'][0]['y']/float(result['image']['height'])} {result['predictions'][0]['width']/float(result['image']['width'])} {result['predictions'][0]['height']/float(result['image']['height'])}\n")
        fp.writelines(preds)
        fp.close()
    else:
        srange_images.append(path)

100%|██████████| 170/170 [03:07<00:00,  1.10s/it]


In [108]:
len(srange_images)

108

In [110]:
for i in srange_images:
    os.remove(i)

Вручную размечаем оставшиеся из srange_images (Косуля_часть2.zip), удаляем папку train

In [111]:
!unzip "/home/jupyter/datasphere/project/олени/Косуля_часть2.zip"

Archive:  /home/jupyter/datasphere/project/олени/Косуля_часть2.zip
 extracting: train/images/-WA0046_jpg.rf.d4b3757e3edc501e0704fb68807dbd36.jpg  
 extracting: train/images/-WA0052_jpg.rf.d189f037aac99e15cb7710e9859e1bdb.jpg  
 extracting: train/images/-WA0053_jpg.rf.800ef8abba7ed765a45724eeee120783.jpg  
 extracting: train/images/Im_0004727_jpg.rf.1ae68aff023430e59ddfd606f767c362.jpg  
 extracting: train/images/Im_0004728_jpg.rf.d6ccbfa960b173e65d90a6d6c3213818.jpg  
 extracting: train/images/Im_0004759_jpg.rf.7c2f640bd979e7b25728f766f79f0c6e.jpg  
 extracting: train/images/Im_0004800_jpg.rf.a4daabbbce01af202062eb1c64a84943.jpg  
 extracting: train/images/Im_0004803_jpg.rf.9c43d6b9ef88463463ae9da890d4bd9c.jpg  
 extracting: train/images/Im_0004809_jpg.rf.efa7dcea5b084052ca7ad00c25a45ecc.jpg  
 extracting: train/images/Im_0004814_jpg.rf.4d2f09ac512560ba3ef71d3819d28050.jpg  
 extracting: train/images/Im_0004837_1_jpg.rf.2cdc3f0033c18242c53b44f65ff12e17.jpg  
 extracting: train/images/I

replace README.roboflow.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


Exception: Process exited with code 1

In [112]:
images = glob.glob('/home/jupyter/datasphere/project/train/images/*')
labels = glob.glob('/home/jupyter/datasphere/project/train/labels/*')

In [114]:
for i in range(len(images)):
    os.rename(images[i], '/home/jupyter/datasphere/project/ndata/images/' + images[i].split('/')[-1])

In [115]:
for i in range(len(labels)):
    os.rename(labels[i], '/home/jupyter/datasphere/project/ndata/labels/' + labels[i].split('/')[-1])

In [116]:
images = glob.glob('/home/jupyter/datasphere/project/ndata/images/*')
labels = glob.glob('/home/jupyter/datasphere/project/ndata/labels/*')

In [118]:
for i in range(len(images)):
    os.rename(images[i], images[i].rstrip(images[i].split('/')[-1])+'kosul_'+images[i].split('/')[-1]) # добавляем префикс для класса 1
for i in range(len(labels)):
    os.rename(labels[i], labels[i].rstrip(labels[i].split('/')[-1])+'kosul_'+labels[i].split('/')[-1]) # добавляем префикс для класса 1

In [119]:
images = glob.glob('/home/jupyter/datasphere/project/ndata/images/*')
labels = glob.glob('/home/jupyter/datasphere/project/ndata/labels/*')

In [121]:
for i in range(len(images)):
    os.rename(images[i], '/home/jupyter/datasphere/project/data/images/' + images[i].split('/')[-1])

In [122]:
for i in range(len(labels)):
    os.rename(labels[i], '/home/jupyter/datasphere/project/data/labels/' + labels[i].split('/')[-1])

Также добавляем в датасет файл Косуля_часть3.zip

## Олень

Создаем папки ddata/images, ddata/labels

In [ ]:
!unzip "/home/jupyter/datasphere/project/олени/Олень.zip" -d "/home/jupyter/datasphere/project/ddata/images"

In [ ]:
kab_paths = glob.glob('ddata/images/Олень/*')

In [ ]:
for i in kab_paths:
    os.rename(i, '/home/jupyter/datasphere/project/ddata/images/' + i.split('/')[-1])

Удаляем папку "Олень"

In [ ]:
images = glob.glob('ddata/images/*')
dest_path = 'ddata/labels'

In [ ]:
images_paths = images[:]

In [ ]:
srange_images = []
for path in tqdm(images_paths):
    if path.endswith('.csv'):
        continue
    if path.endswith('ОлениPTDC0046.JPG'):
        os.remove(path)
        continue
    result = model.predict(path, confidence=30, overlap=50).json()
    name = path.split('/')[-1].split('.')[0]
    if len(result['predictions']) > 0:
        fp = open(f'{dest_path}/{name}.txt', 'w')
        preds = []
        for pred in result['predictions']:
            preds.append(f"2 {result['predictions'][0]['x']/float(result['image']['width'])} {result['predictions'][0]['y']/float(result['image']['height'])} {result['predictions'][0]['width']/float(result['image']['width'])} {result['predictions'][0]['height']/float(result['image']['height'])}\n")
        fp.writelines(preds)
        fp.close()
    else:
        srange_images.append(path)

In [ ]:
for i in srange_images:
    os.remove(i)
os.remove('/home/jupyter/datasphere/project/ddata/images/Благородныйолень.csv')

Вручную размечаем оставшиеся из srange_images (Олень_часть2.zip), удаляем папку train

In [ ]:
!unzip "/home/jupyter/datasphere/project/олени/Олень_часть2.zip"

In [ ]:
images = glob.glob('/home/jupyter/datasphere/project/train/images/*')
labels = glob.glob('/home/jupyter/datasphere/project/train/labels/*')

In [ ]:
for i in range(len(images)):
    os.rename(images[i], '/home/jupyter/datasphere/project/ddata/images/' + images[i].split('/')[-1])

In [ ]:
for i in range(len(labels)):
    os.rename(labels[i], '/home/jupyter/datasphere/project/ddata/labels/' + labels[i].split('/')[-1])

In [ ]:
images = glob.glob('/home/jupyter/datasphere/project/ddata/images/*')
labels = glob.glob('/home/jupyter/datasphere/project/ddata/labels/*')

In [ ]:
for i in range(len(images)):
    os.rename(images[i], images[i].rstrip(images[i].split('/')[-1])+'olen_'+images[i].split('/')[-1]) # добавляем префикс для класса 2

In [ ]:
for i in range(len(labels)):
    os.rename(labels[i], labels[i].rstrip(labels[i].split('/')[-1])+'olen_'+labels[i].split('/')[-1]) # добавляем префикс для класса 2

In [ ]:
images = glob.glob('/home/jupyter/datasphere/project/ddata/images/*')
labels = glob.glob('/home/jupyter/datasphere/project/ddata/labels/*')

In [ ]:
for i in range(len(images)):
    os.rename(images[i], '/home/jupyter/datasphere/project/data/images/' + images[i].split('/')[-1])

In [ ]:
for i in range(len(labels)):
    os.rename(labels[i], '/home/jupyter/datasphere/project/data/labels/' + labels[i].split('/')[-1])

Также добавляем в датасет файл Олень_часть3.zip

## Train/valid

In [142]:
images = glob.glob('/home/jupyter/datasphere/project/data/images/*')
labels = glob.glob('/home/jupyter/datasphere/project/data/labels/*')

In [143]:
len(images), len(labels)

(7134, 7132)

In [139]:
images_set = set(['.'.join(i.split('/')[-1].split('.')[:-1]) for i in images])
labels_set = set(['.'.join(i.split('/')[-1].split('.')[:-1]) for i in labels])

In [140]:
labels_set.difference(images_set)

{'cabarga_1920x1280_1541485_[www',
 'cabarga_1920x1280_1541636_[www',
 'cabarga_1920x1315_1543436_[www',
 'cabarga_2000x1333_1542021_[www',
 'cabarga_2133x1200_1542933_[www',
 'cabarga_Akron Zoo 2018',
 'cabarga_Feb',
 'cabarga_o',
 'cabarga_ris',
 'cabarga_tr2_musk deer_14',
 'cabarga_tr30a_muskDeer_31',
 'cabarga_tr47_muskDeer_02',
 'cabarga_tr47_muskDeer_12',
 'cabarga_tr47_muskDeer_18',
 'cabarga_tr49a_muskDeer_06',
 'cabarga_tr50_MuskDeer_24',
 'cabarga_tr50_MuskDeer_27',
 'cabarga_tr55_MusckDeer_22',
 'cabarga_tr56_muskDeer_26',
 'cabarga_tr58_musckDeer_06',
 'cabarga_tr62_MuskDeer_23',
 'cabarga_tr62_RoeDeer_25',
 'cabarga_tr63_muskDeer_28',
 'cabarga_tr64a_MuskDeer_20',
 'kosul_roeDeer_48_07',
 'kosul_roeDeer_58_26',
 'kosul_tr18a_roe deer_25',
 'kosul_tr18a_roe deer_3',
 'kosul_tr23_roeDeer_05',
 'kosul_tr36_roeDeer_02',
 'kosul_tr36_roeDeer_20',
 'kosul_tr48a_RoeDeer_04',
 'kosul_tr48a_roeDeer_23',
 'kosul_tr49a_roeDeer_17',
 'kosul_tr50_roeDeer_28',
 'kosul_tr51_roeDeer_01',

In [141]:
for i in list(labels_set.difference(images_set)):
    os.remove(f'/home/jupyter/datasphere/project/data/labels/{i}.txt')


In [10]:
import os
import random
import shutil
from glob import glob

images = glob("/home/jupyter/datasphere/project/data/images/*")

d = {"olen" :[], "kosul" : [], "cabarga" :[]}

for i in images:
    for j in d.keys():
        if i.split("/")[-1].startswith(j):
            d[j].append(i)
            break
            
        

In [9]:
len(d["kosul"]),len(d["olen"]),len(d["cabarga"])

(2368, 2778, 1429)

In [4]:
for key in d.keys():
    
    val_size = int(len(d[key]) * 0.2)
    
    val_filenames = set(random.sample(d[key], val_size))
    train_filenames = set(d[key]) - val_filenames
    
    for i in val_filenames:
        try:
            shutil.copy(i.rstrip(i.split('.')[-1]).replace("images", "labels",1)+'txt', "/home/jupyter/datasphere/project/data_yolo2/labels/val/" + i.split('/')[-1].rstrip(i.split('.')[-1])+'txt')
            shutil.copy(i, "/home/jupyter/datasphere/project/data_yolo2/images/val/" + i.split('/')[-1])
        except Exception:
            continue
    
    for i in train_filenames:
        try:
            shutil.copy(i.rstrip(i.split('.')[-1]).replace("images", "labels",1)+'txt', "/home/jupyter/datasphere/project/data_yolo2/labels/train/" + i.split('/')[-1].rstrip(i.split('.')[-1])+'txt')
            shutil.copy(i, "/home/jupyter/datasphere/project/data_yolo2/images/train/" + i.split('/')[-1])
        except Exception:
            continue